In [13]:
import sys
import pandas as pd
import numpy as np

sys.path.append('../function/')
from function import test

# 14.1 Simulate Data

I skip this part because it's easy and I can't generate the exactly same random number as in R.

I'll import the data from assignment in R for the rest part.

In [56]:
# set the seed
np.random.seed(1)
# number of products
J = 10
# dimension of product characteristics including the intercept
K = 3
# number of markets
T = 100
# number of consumers per market
N = 500
# number of Monte Carlo
L = 500

# set parameters of interests
beta = [4.0000000, 0.1836433, -0.8356286]
print(f'beta is {beta}')

sigma = [1.5952808, 0.3295078, 0.8204684]
print(f'sigma is {sigma}')

mu = 0.5
omega = 1

# set auxiliary parameters
price_xi = 1
sd_x = 2
sd_xi = 0.1
sd_c = 0.5
sd_p = 0.01

beta is [4.0, 0.1836433, -0.8356286]
sigma is [1.5952808, 0.3295078, 0.8204684]


In [75]:
V = pd.read_csv('data/V.csv')
X = pd.read_csv('data/X.csv')
M = pd.read_csv('data/M.csv')
df = pd.read_csv('data/df.csv')
df_share_smooth = pd.read_csv('data/df_share_smooth.csv')
df_choice_smooth = pd.read_csv('data/df_choice_smooth.csv')
V_mcmc = pd.read_csv('data/V_mcmc.csv')
e_mcmc = pd.read_csv('data/e_mcmc.csv')
beta = np.array(pd.read_csv('data/beta.csv')).reshape(-1,)
sigma = np.array(pd.read_csv('data/sigma.csv')).reshape(-1,)

# 14.2 Estimate the parameters

## 14.2.1

In [61]:
#import V_mcmc from R
V_mcmc

,i,t,v_x_1,v_x_2,v_x_3,v_p
0,1,1,0.488092,-1.508735,0.528268,-0.468416
1,2,1,1.157528,0.506822,-0.527316,-0.515559
2,3,1,-2.494002,-0.318272,-0.099627,-0.893371
3,4,1,0.095167,-0.133354,-2.050517,1.918877
4,5,1,-1.113310,0.103226,2.243173,0.753177
...,...,...,...,...,...,...
49995,496,100,-0.819919,-2.236706,-0.443720,0.167854
49996,497,100,0.137973,-0.702396,0.937442,0.385176
49997,498,100,0.568157,-2.325619,1.246583,0.639265
49998,499,100,0.586307,-1.054256,-0.559406,-0.979268


In [74]:
df_mcmc = pd.DataFrame([(t,i,j) for t in range(1, T+1) for i in range(1, L+1) for j in range(J+1)])
df_mcmc.columns = ['t', 'i', 'j']
df_mcmc = pd.merge(df_mcmc, V_mcmc, how='left', on=['i', 't'])
df_mcmc = pd.merge(df_mcmc, X, how='left', on=['j'])
df_mcmc = pd.merge(df_mcmc, M, how='left', on=['j', 't'])
df_mcmc = df_mcmc.dropna(subset = ['p']).sort_values(by=['t', 'i', 'j'], ascending=True)
df_mcmc

,t,i,j,v_x_1,v_x_2,v_x_3,v_p,x_1,x_2,x_3,xi,c,p
0,1,1,0,0.488092,-1.508735,0.528268,-0.468416,0,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,1,0.488092,-1.508735,0.528268,-0.468416,1,0.974858,-0.032381,-0.015580,0.603655,1.584006
4,1,1,4,0.488092,-1.508735,0.528268,-0.468416,1,-0.610777,1.187803,0.041794,1.301829,2.352555
11,1,2,0,1.157528,0.506822,-0.527316,-0.515559,0,0.000000,0.000000,0.000000,0.000000,0.000000
12,1,2,1,1.157528,0.506822,-0.527316,-0.515559,1,0.974858,-0.032381,-0.015580,0.603655,1.584006
...,...,...,...,...,...,...,...,...,...,...,...,...,...
549989,100,500,0,0.254454,2.145978,0.868148,-1.483094,0,0.000000,0.000000,0.000000,0.000000,0.000000
549990,100,500,1,0.254454,2.145978,0.868148,-1.483094,1,0.974858,-0.032381,-0.059143,0.731577,1.667251
549991,100,500,2,0.254454,2.145978,0.868148,-1.483094,1,1.476649,1.887672,-0.133403,0.645601,1.496741
549992,100,500,3,0.254454,2.145978,0.868148,-1.483094,1,1.151563,1.642442,-0.109730,1.074310,1.967498


In [69]:
#import e_mcmc from R
e_mcmc = np.array(e_mcmc).reshape(-1,)
e_mcmc[:6]

array([ 0.10060134,  2.70398241,  1.05402778,  2.46973885,  1.6721181 ,
       -1.02838718])

## 14.2.2

In [79]:
theta = list(beta) + list(sigma) + [mu] + [omega]
theta

[4.0,
 0.183643324222082,
 -0.835628612410047,
 1.59528080213779,
 0.329507771815361,
 0.820468384118015,
 0.5,
 1]

## 14.2.3

In [78]:
list([1])

[1]

### 14.1.2

In [54]:
# make product characteristics data
X = (sd_x * np.random.normal(0, 1, J * (K - 1))).reshape((J, K-1))
X = np.concatenate((np.ones((J,1)), X), axis=1)
X = np.concatenate((np.arange(1, J+1).reshape(-1,1), X), axis=1)
X = np.concatenate((np.zeros(K+1).reshape(1,-1), X), axis=0)
X = pd.DataFrame(X)
col_name = ['j'] + [ 'x_' + str(i) for i in range(1, K+1)]
X.columns = col_name
X

,j,x_1,x_2,x_3
0,0.0,0.0,0.000000,0.000000
1,1.0,1.0,-1.073259,0.630805
2,2.0,1.0,0.842101,-2.131206
3,3.0,1.0,-1.772479,-0.951467
4,4.0,1.0,1.379365,1.122384
5,5.0,1.0,-2.611097,-2.238951
6,6.0,1.0,1.473675,3.149268
7,7.0,1.0,-0.062150,-1.366893
8,8.0,1.0,2.191259,-0.619153
9,9.0,1.0,1.451504,3.098143


### 14.1.3

In [55]:
[(j,t) for j in range(1, J+1) for t in range(1, T+1)]

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (1, 25),
 (1, 26),
 (1, 27),
 (1, 28),
 (1, 29),
 (1, 30),
 (1, 31),
 (1, 32),
 (1, 33),
 (1, 34),
 (1, 35),
 (1, 36),
 (1, 37),
 (1, 38),
 (1, 39),
 (1, 40),
 (1, 41),
 (1, 42),
 (1, 43),
 (1, 44),
 (1, 45),
 (1, 46),
 (1, 47),
 (1, 48),
 (1, 49),
 (1, 50),
 (1, 51),
 (1, 52),
 (1, 53),
 (1, 54),
 (1, 55),
 (1, 56),
 (1, 57),
 (1, 58),
 (1, 59),
 (1, 60),
 (1, 61),
 (1, 62),
 (1, 63),
 (1, 64),
 (1, 65),
 (1, 66),
 (1, 67),
 (1, 68),
 (1, 69),
 (1, 70),
 (1, 71),
 (1, 72),
 (1, 73),
 (1, 74),
 (1, 75),
 (1, 76),
 (1, 77),
 (1, 78),
 (1, 79),
 (1, 80),
 (1, 81),
 (1, 82),
 (1, 83),
 (1, 84),
 (1, 85),
 (1, 86),
 (1, 87),
 (1, 88),
 (1, 89),
 (1, 90),
 (1, 91),
 (1, 92),
 (1, 93),
 (1, 94),
 (1, 95),
 (1, 96),
 (1, 97),
 (1, 98),
 (1, 99),
 (1, 100),
 (2, 1),

In [ ]:
# make market-product data
M = 

In [ ]:
# make market-product data
M <- 
  expand.grid(
    j = 1:J, 
    t = 1:T
    ) %>%
    tibble::as_tibble() %>%
    dplyr::mutate(
      xi = sd_xi * rnorm(J * T),
      c = exp(sd_c * rnorm(J * T)),
      p = exp(price_xi * xi + sd_p * rnorm(J * T)) + c
    ) 
M <- 
  M %>%
  dplyr::group_by(t) %>%
  dplyr::sample_frac(size = purrr::rdunif(1, J) / J) %>%
  dplyr::ungroup()
# add outside option
outside <- 
  data.frame(
    j = 0, 
    t = 1:T, 
    xi = 0, 
    c = 0, 
    p = 0
    )
M <- 
  rbind(
    M,
    outside
    ) %>%
  dplyr::arrange(
    t, 
    j
    )
M

In [28]:
X

array([[ 1.46454271, -1.28515078],
       [-0.35618635, -1.14790912],
       [-0.40875063, -0.9729902 ],
       [-0.37155065, -0.76107283],
       [ 0.17795527,  0.12734332],
       [ 0.59269422,  2.80554224],
       [-3.09372514,  2.59123717],
       [-0.47450089, -2.46469241],
       [-0.34483953,  0.18367675],
       [ 2.13511691, -2.1232689 ]])

In [51]:
np.arange(1, J+1).reshape(1,-1)

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])

In [30]:
X = np.concatenate((np.ones((J,1)), X), axis=1)